In [29]:
# Crank-Nicolson Method
import numpy 
import  matplotlib.pyplot as  pyplot
import time
tic = time.clock()
numpy.set_printoptions(precision=6)


In [30]:
# Specifiy the geometry of the domain

L = 22.02*10**(-3) # total length of the channel (m), it was 15.5 mm
J = 701 # number of grids in space, 40um/grid (J=176), 10um/grid (J=701)
#dx = float(L)/float(J-1)
dx = 1.0e-5 # coarse grid size
x_grid = numpy.array([j*dx for j in range(J)]) #source

mem_loc = [7*10**(-3),7.01*10**(-3),15.01*10**(-3),15.02*10**(-3)]
#mem_loc_index = round(float(mem_loc)/float(dx))
J1 = numpy.linspace(mem_loc[0],mem_loc[1],num=2) # grids within 1st membrane 
x_grid = numpy.concatenate((x_grid[0:-1],J1[0:-1])) 
gap_grid=numpy.linspace(mem_loc[1],mem_loc[2],num=801) # grids between 1st and 2nd membrane for 40 um/grid size (num = 26 for 1 mm, 51 for 2 mm, 101 for 4mm, 201 for 8mm)
x_grid=numpy.concatenate((x_grid,gap_grid[0:-1]))# gap
J2 = numpy.linspace(mem_loc[2],mem_loc[3],num=2) # grids within 2nd membrane 
#x_grid = numpy.array([j*dx for j in range(J)])
x_grid = numpy.concatenate((x_grid,J2[0:-1]))
drain_grid=numpy.linspace(mem_loc[3],L,num=701) # for 40 um/grid size  (num=188 for 1 mm, 163 for 2mm, 113 for 4 mm, 13 for 8mm)
x_grid=numpy.concatenate((x_grid,drain_grid))#drain

#x_grid = numpy.sort(x_grid, axis=None)
#x_grid.append(J1)
#x_grid.appedn(J2)
#x_grid.sort()
T = 120*60 # total time (sec) 
N = 1000 # number of grids in time
dt = float(T)/float(N-1)
t_grid = numpy.array([n*dt for n in range(N)])

pyplot.scatter(x_grid,x_grid)
pyplot.show()

In [31]:
# Define the diffusion coefficient of the chamber and membrane

d_u = 4.*5.5*10**(-10) # Alpha Methyl Aspartate (m2/s)
d_m = 0.03*d_u # touluene diffucivity at the membranes
#D_c =  (D_u+D_m)/2 # centering the diffusion coefficient 
d_c = (dx+(J1[1]-J1[0]))/2 # centering the grid size

sigma_u = float(d_u*dt)/float((2.*dx*dx)) 
sigma_m = float(d_m*dt)/float((2.*(J1[1]-J1[0])*(J1[1]-J1[0])))
#sigma_c = float(D_c*dt)/float((2.*d_c*d_c))

In [32]:
# specify the initial condition for attractant

no_high = numpy.where(x_grid == mem_loc[0]) # location of the 1st membrane
no_high = numpy.asarray(no_high[0])
second_mem = numpy.where(x_grid == mem_loc[2]) # location of the 2nd membrane
second_mem = numpy.asarray(second_mem[0])
U =  numpy.array([1. for i in range(0,no_high)]+ [0. for i in range(no_high,len(x_grid))]) 
print no_high
print second_mem
#fig1 = pyplot.figure()
#pyplot.ylim((0., 1.))
#pyplot.xlabel('x[m]'); pyplot.ylabel('concentration[mM]')
#pyplot.plot(x_grid, U)
#pyplot.show()
#fig1.savefig('Initial_Repellent_concentration.png')

[700]
[1501]


In [33]:
#Create Matrices
#The matrices that we need to construct are all tridiagonal so they are easy to construct with 
#numpy.diagflat.

# it was for i in range(J-1)
A_u = numpy.diagflat([-sigma_u for i in range(len(x_grid)-1)], -1) +\
      numpy.diagflat([1.+sigma_u]+[1.+2.*sigma_u for i in range(len(x_grid)-2)]+[1.+sigma_u]) +\
      numpy.diagflat([-sigma_u for i in range(len(x_grid)-1)], 1)
        
B_u = numpy.diagflat([sigma_u for i in range(len(x_grid)-1)], -1) +\
      numpy.diagflat([1.-sigma_u]+[1.-2.*sigma_u for i in range(len(x_grid)-2)]+[1.-sigma_u]) +\
      numpy.diagflat([sigma_u for i in range(len(x_grid)-1)], 1)
        
# 1st membrane left boundary
sigma_c_minus = float(d_u*dt)/float(2.*dx*d_c) # outside membrane 
sigma_c_plus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane

A_u[no_high,no_high]=1.+sigma_c_plus+sigma_c_minus
A_u[no_high,no_high-1]=-sigma_c_minus
A_u[no_high,no_high+1]=-sigma_c_plus
B_u[no_high,no_high]=1.-sigma_c_plus-sigma_c_minus
B_u[no_high,no_high-1]=sigma_c_minus
B_u[no_high,no_high+1]=sigma_c_plus

#within the 1st membrane only 
if len(J1) >= 3: # only work if there's more than 3 grids in the membrane
    for i in range(len(J1)-2): # exclude the left and right boundary
        i = i+1 # next of the left boundary
        A_u[no_high+i,no_high+i]=1.+2.*sigma_m
        A_u[no_high+i,no_high+i-1]=-sigma_m
        A_u[no_high+i,no_high+i+1]=-sigma_m
        B_u[no_high+i,no_high+i]=1.-2.*sigma_m
        B_u[no_high+i,no_high+i-1]=sigma_m
        B_u[no_high+i,no_high+i+1]=sigma_m

# 1st membrane right boundary
sigma_c_minus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane 
sigma_c_plus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane

A_u[no_high+2,no_high+2]=1.+sigma_c_plus+sigma_c_minus
A_u[no_high+2,no_high+2-1]=-sigma_c_minus
A_u[no_high+2,no_high+2+1]=-sigma_c_plus
B_u[no_high+2,no_high+2]=1.-sigma_c_plus-sigma_c_minus
B_u[no_high+2,no_high+2-1]=sigma_c_minus
B_u[no_high+2,no_high+2+1]=sigma_c_plus
    
# 2nd membrane left boundary
sigma_c_minus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane 
sigma_c_plus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane

A_u[second_mem,second_mem]=1.+sigma_c_plus+sigma_c_minus
A_u[second_mem,second_mem-1]=-sigma_c_minus
A_u[second_mem,second_mem+1]=-sigma_c_plus
B_u[second_mem,second_mem]=1.-sigma_c_plus-sigma_c_minus
B_u[second_mem,second_mem-1]=sigma_c_minus
B_u[second_mem,second_mem+1]=sigma_c_plus

# within the 2nd membrane 
if len(J2) >=3: # work only if there's more than 3 elements in J2
    for i in range(len(J2)-2): # excluding the left and right boundary
        i = i+1 # next of the left boundary
        A_u[second_mem+i,second_mem+i]=1.+2.*sigma_m
        A_u[second_mem+i,second_mem+i-1]=-sigma_m
        A_u[second_mem+i,second_mem+i+1]=-sigma_m
        B_u[second_mem+i,second_mem+i]=1.-2.*sigma_m
        B_u[second_mem+i,second_mem+i-1]=sigma_m
        B_u[second_mem+i,second_mem+i+1]=sigma_m

# 2nd membrane right boundary
sigma_c_minus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane 
sigma_c_plus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane

A_u[second_mem+2,second_mem+2]=1.+sigma_c_plus+sigma_c_minus
A_u[second_mem+2,second_mem+2-1]=-sigma_c_minus
A_u[second_mem+2,second_mem+2+1]=-sigma_c_plus
B_u[second_mem+2,second_mem+2]=1.-sigma_c_plus-sigma_c_minus
B_u[second_mem+2,second_mem+2-1]=sigma_c_minus
B_u[second_mem+2,second_mem+2+1]=sigma_c_plus  


In [34]:
#To confirm, this is what A_u, B_u looks like:    

print A_u
print A_u[no_high[0],no_high[0]]
print A_u[no_high[0],no_high[0]-1]
print A_u[no_high[0],no_high[0]+1]
print A_u[second_mem[0],second_mem[0]]
print A_u[second_mem[0],second_mem[0]-1]
print A_u[second_mem[0],second_mem[0]+1]

print B_u[no_high[0],no_high[0]]
print B_u[no_high[0],no_high[0]-1]
print B_u[no_high[0],no_high[0]+1]
print B_u[second_mem[0],second_mem[0]]
print B_u[second_mem[0],second_mem[0]-1]
print B_u[second_mem[0],second_mem[0]+1]

[[  80.279279  -79.279279    0.       ...,    0.          0.          0.      ]
 [ -79.279279  159.558559  -79.279279 ...,    0.          0.          0.      ]
 [   0.        -79.279279  159.558559 ...,    0.          0.          0.      ]
 ..., 
 [   0.          0.          0.       ...,  159.558559  -79.279279    0.      ]
 [   0.          0.          0.       ...,  -79.279279  159.558559
   -79.279279]
 [   0.          0.          0.       ...,    0.        -79.279279
    80.279279]]
82.6576576577
-79.2792792793
-2.37837837838
82.6576576577
-79.2792792793
-2.37837837838
-80.6576576577
79.2792792793
2.37837837838
-80.6576576577
79.2792792793
2.37837837838


In [35]:
# Solve the system iteratively
U_record = []

U_record.append(U)

for ti in range(1,N):
    U_new = numpy.linalg.solve(A_u,B_u.dot(U))    
    U = U_new 
    U_record.append(U)



In [36]:
# Plot the numerical solution at the last time snap shot
fig2 = pyplot.figure()
pyplot.ylim((0., 1.))
pyplot.xlabel('x[m]'); pyplot.ylabel('concentration[mM]')
pyplot.plot(x_grid, U)
pyplot.show()
fig2.savefig('Transient_repellent_gradient.png')



In [25]:
# Plot the heatmap of the chemical concentration
U_record = numpy.array(U_record) # attractant spatial and temporal data
fig3, ax = pyplot.subplots()
pyplot.xlabel('x[m]'); pyplot.ylabel('t[sec]')
heatmap = ax.pcolor(x_grid, t_grid, U_record, vmin=0., vmax=2.1)
colorbar = pyplot.colorbar(heatmap)
pyplot.show()
fig3.savefig('Repellent_transient_heatmap.png')

In [37]:
# Calculate V_C, advection coefficient
mu = 28.0*10**(-6) # bacterial swimming speed (m/s) E-coli
ki = 5.0*10**(-8) # m2/s
kc = 0.125 # mM, mol/m3
mu =44.0*10**(-6) # bacterial swimming speed (m/s)                                                         

r = len(t_grid) # length of the temporal grids
c = len(x_grid) # length of the spatial grids
V_C = numpy.zeros([r,c])
# U_record is a list, and needs to convert it to an array
U_record = numpy.asarray(U_record)

V_C[:,0] =2./3.*mu*numpy.tanh(ki/(2.*mu)*kc/((kc+U_record[:,0])**2)*
                              (U_record[:,1]-U_record[:,0])/(x_grid[j+1]-x_grid[j])) 

for n in range(0,r): # time
    for j in range(0,c-1): # space
        V_C[n, j+1] = 2./3.*mu*numpy.tanh(ki/(2.*mu)*kc/((kc+U_record[n,j])**2)*
                                          (U_record[n,j+1]-U_record[n,j])
                                          /(x_grid[j+1]-x_grid[j]))
        

#print [V_C[0,i] for i in range(5)]

V_C_d = numpy.zeros([r,c]) # the derivative of V_C using Crank-Nicolson method

# left boundary, first column, c =0 

for n in range(0,r-1): # time  
    V_C_d[n,0] = 1./(4.*x_grid[1]-x_grid[0])*(V_C[n,1]-V_C[n,0]+V_C[n+1,1]-V_C[n+1,0])

# right boundary,last column, c-1
 
for n in range(0,r-1): # time  
    V_C_d[n,c-1] = 1./(4.*x_grid[c-1]-x_grid[c-2])*(V_C[n,c-1]-V_C[n,c-2]+V_C[n+1,c-1]-V_C[n+1,c-2])
 

# the rest of the matrix
for n in range(0,r-1): # time 
    for j in range(1,c-2): # space, excluding the first and last column
        V_C_d[n,j] = 1./(4.*(x_grid[j+1]-x_grid[j]))*(V_C[n,j+1]-V_C[n,j-1]+V_C[n+1,j+1]-V_C[n+1,j-1])



In [42]:
# Plot V_C_d
figV_C_d = pyplot.figure()
heatmap = pyplot.pcolor(V_C_d)
colorbar = pyplot.colorbar(heatmap)
pyplot.show() 
figV_C_d.savefig('V_C_d heatmap.png')


In [34]:
# Plot V_C
fig4 = pyplot.figure()
heatmap = pyplot.pcolor(V_C)
colorbar = pyplot.colorbar(heatmap)
pyplot.show() 
fig4.savefig('V_C heatmap.png')


In [38]:
# Calculate the bacterial population

# initial condition of the bacteria
no_high = numpy.where(x_grid == mem_loc[1]) # location of the 1st membrane                
no_high = no_high[0]
second_mem = numpy.where(x_grid == mem_loc[2]) # location of the 2nd membrane                              
second_mem = second_mem[0]
B =  numpy.array([0. for i in range(0,no_high+1)]+[100. for i in range(no_high+1, second_mem)]+
                 [0. for i in range(second_mem,len(x_grid))]) # step function initial condition                 

fig5 = pyplot.figure()
pyplot.ylim((0., 100.))
pyplot.xlabel('x[m]'); pyplot.ylabel('Bacterial Population[AU]')
pyplot.plot(x_grid, B)
pyplot.show()
fig5.savefig('Initial_Bacterial_Population.png')


In [39]:
# Construct Bacterial Diffusion Equation

# it was for i in range(J-1)
d_u = 5.9*10**(-10) # E-coli motility (m2/s)
d_m = 0.03*d_u # E-coli diffucivity at the membranes
d_c = (dx+(J1[1]-J1[0]))/2 # averaging the coarse and fine grid sizes
rho = float(dt)/float(4.*dx) 

print d_c
print J1[1]-J1[0]
sigma_u = float(d_u*dt)/float((2.*dx*dx)) 
sigma_m = float(d_m*dt)/float((2.*(J1[1]-J1[0])*(J1[1]-J1[0])))
C_u=[]
D_u=[]

def CuDu_update(V_C_I,V_C_d_I): 
    C_u = numpy.diagflat([-sigma_u+rho*V_C_I[i] for i in range(1,len(x_grid))], -1) + \
      numpy.diagflat([1.+sigma_u+rho*V_C_I[0]]+[1.+2.*sigma_u for i in range(len(x_grid)-2)] + \
                     [1.+sigma_u-rho*V_C_I[len(x_grid)-1]]) + \
      numpy.diagflat([-(sigma_u+rho*V_C_I[i]) for i in range(1,len(x_grid))], 1)
        
    D_u = numpy.diagflat([sigma_u-rho*V_C_I[i] for i in range(1,len(x_grid))], -1) + \
      numpy.diagflat([1.-sigma_u-rho*V_C_I[0]+V_C_d_I[0]]+[1.-2.*sigma_u for i in range(len(x_grid)-2)]+ \
                     [1.-sigma_u+V_C_d_I[len(x_grid)-1]+rho*(V_C_I[len(x_grid)-1])]) + \
      numpy.diagflat([sigma_u+rho*(V_C_I[i]) for i in range(1,len(x_grid))], 1)

# 1st membrane left boundary
    sigma_c_minus = float(d_u*dt)/float(2.*dx*d_c) # outside membrane 
    sigma_c_plus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane

    C_u[no_high,no_high]=1.+sigma_c_plus+sigma_c_minus
    C_u[no_high,no_high-1]=-sigma_c_minus+rho*V_C_I[no_high]
    C_u[no_high,no_high+1]=-sigma_c_plus-rho*V_C_I[no_high]
    D_u[no_high,no_high]=1.-sigma_c_plus-sigma_c_minus
    D_u[no_high,no_high-1]=sigma_c_minus-rho*V_C_I[no_high]
    D_u[no_high,no_high+1]=sigma_c_plus+rho*V_C_I[no_high]

#within the 1st membrane only (not yet updated)
    if len(J1) >= 3: # only work if there's more than 3 grids in the membrane
        for i in range(len(J1)-2): # exclude the left and right boundary
            i = i+1 # next of the left boundary
            C_u[no_high+i,no_high+i]=1.+2.*sigma_m
            C_u[no_high+i,no_high+i-1]=-sigma_m
            C_u[no_high+i,no_high+i+1]=-sigma_m
            D_u[no_high+i,no_high+i]=1.-2.*sigma_m
            D_u[no_high+i,no_high+i-1]=sigma_m
            D_u[no_high+i,no_high+i+1]=sigma_m

# 1st membrane right boundary
    sigma_c_minus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane 
    sigma_c_plus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane

    C_u[no_high+2,no_high+2]=1.+sigma_c_plus+sigma_c_minus
    C_u[no_high+2,no_high+2-1]=-sigma_c_minus+rho*V_C_I[no_high+2]
    C_u[no_high+2,no_high+2+1]=-sigma_c_plus-rho*V_C_I[no_high+2]
    D_u[no_high+2,no_high+2]=1.-sigma_c_plus-sigma_c_minus
    D_u[no_high+2,no_high+2-1]=sigma_c_minus-rho*V_C_I[no_high+2]
    D_u[no_high+2,no_high+2+1]=sigma_c_plus+rho*V_C_I[no_high+2]
    
# 2nd membrane left boundary
    sigma_c_minus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane 
    sigma_c_plus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane

    C_u[second_mem,second_mem]=1.+sigma_c_plus+sigma_c_minus
    C_u[second_mem,second_mem-1]=-sigma_c_minus+rho*V_C_I[second_mem]
    C_u[second_mem,second_mem+1]=-sigma_c_plus-rho*V_C_I[second_mem]
    D_u[second_mem,second_mem]=1.-sigma_c_plus-sigma_c_minus
    D_u[second_mem,second_mem-1]=sigma_c_minus-rho*V_C_I[second_mem]
    D_u[second_mem,second_mem+1]=sigma_c_plus-rho*V_C_I[second_mem]

# within the 2nd membrane (not yet update)
    if len(J2) >=3: # work only if there's more than 3 elements in J2
        for i in range(len(J2)-2): # excluding the left and right boundary
            i = i+1 # next of the left boundary
            C_u[second_mem+i,second_mem+i]=1.+2.*sigma_m
            C_u[second_mem+i,second_mem+i-1]=-sigma_m
            C_u[second_mem+i,second_mem+i+1]=-sigma_m
            D_u[second_mem+i,second_mem+i]=1.-2.*sigma_m
            D_u[second_mem+i,second_mem+i-1]=sigma_m
            D_u[second_mem+i,second_mem+i+1]=sigma_m

# 2nd membrane right boundary
    sigma_c_minus = float(d_m*dt)/float((2.*(J1[1]-J1[0])*d_c)) # within membrane 
    sigma_c_plus = float(d_u*dt)/float((2.*dx*d_c)) # outside membrane

    C_u[second_mem+2,second_mem+2]=1.+sigma_c_plus+sigma_c_minus
    C_u[second_mem+2,second_mem+2-1]=-sigma_c_minus+rho*V_C_I[second_mem+2]
    C_u[second_mem+2,second_mem+2+1]=-sigma_c_plus-rho*V_C_I[second_mem+2]
    D_u[second_mem+2,second_mem+2]=1.-sigma_c_plus-sigma_c_minus
    D_u[second_mem+2,second_mem+2-1]=sigma_c_minus-rho*V_C_I[second_mem+2]
    D_u[second_mem+2,second_mem+2+1]=sigma_c_plus+rho*V_C_I[second_mem+2]
    
    return C_u, D_u

C_u,D_u = CuDu_update(V_C[0,:],V_C_d[0,:])

print C_u
print D_u
# print ('dx is ' + str(dx))
# print ('finer grid is '+ str(J1[1]-J1[0]))
# print('average grid size is '+ str(d_c))
#print ('sigma_m is '+ str(sigma_m))
#print ('1-2.*sigma_c is '+ str(1.-2.*sigma_c))
#print ('sigma_c is '+ str(sigma_c))
# print C_u[no_high,no_high]
# print C_u[no_high,no_high-1]
# print C_u[no_high,no_high+1]
# print C_u[second_mem,second_mem]
# print C_u[second_mem,second_mem-1] 
# print C_u[second_mem,second_mem+1] 
# print D_u[no_high,no_high]
# print D_u[no_high,no_high-1]
# print D_u[no_high,no_high+1]
# print D_u[second_mem,second_mem]
# print D_u[second_mem,second_mem-1] 
# print D_u[second_mem,second_mem+1] 

1e-05
1e-05
[[ 22.261261 -21.261261   0.       ...,   0.         0.         0.      ]
 [-21.261261  43.522523 -21.261261 ...,   0.         0.         0.      ]
 [  0.       -21.261261  43.522523 ...,   0.         0.         0.      ]
 ..., 
 [  0.         0.         0.       ...,  43.522523 -21.261261   0.      ]
 [  0.         0.         0.       ..., -21.261261  43.522523 -21.261261]
 [  0.         0.         0.       ...,   0.       -21.261261  22.261261]]
[[-20.261261  21.261261   0.       ...,   0.         0.         0.      ]
 [ 21.261261 -41.522523  21.261261 ...,   0.         0.         0.      ]
 [  0.        21.261261 -41.522523 ...,   0.         0.         0.      ]
 ..., 
 [  0.         0.         0.       ..., -41.522523  21.261261   0.      ]
 [  0.         0.         0.       ...,  21.261261 -41.522523  21.261261]
 [  0.         0.         0.       ...,   0.        21.261261 -20.261261]]


In [17]:
# initial condition for the f_vec


#r = len(t_grid) # length of the temporal grids
#c = len(x_grid) # length of the spatial grids

#B_record = numpy.zeros([r,c])
#B = numpy.asarray(B)
#B_record[0,:] = B # initial condition

# not yet done
#for i in range(0,r-1): # time
#    for j in range(0,c-1): # space
#        B_record[i+1,j] = -dt*((V_C[i,j+1]-V_C[i,j])/(x_grid[i,j+1]-x_grid[i,j])*B[i,j]+(B[i,j+1]-B[i,j])/(x_grid[i,j+1]-x_grid[i,j])*V_C[i,j])

#for j in range(len(B)): # loop all spatial grids
#    f_vec = numpy.multiply(dt,numpy.add(numpy.multiply(B,numpy.divide(numpy.subtract(V_C[0,j+1],V_C[0,j]),dx)),numpy. multiply(V_C[0,j],numpy.divide(numpy.subtract(B[0,j+1],B[0,j]),dx))))
#############


In [40]:
B_record = []
B_receiver_record = []

B_record.append(B)
B_receiver_record.append(B[(second_mem+1):-1])

for ti in range(1,N):
    V_C_I = V_C[ti-1,:]
    V_C_d_I = V_C_d[ti-1,:]
    C_u,D_u = CuDu_update(V_C_I,V_C_d_I)
    B_new = numpy.linalg.solve(C_u,D_u.dot(B))    
    B = B_new 
    B_record.append(B)
    B_receiver = B[(second_mem+1):-1] # bacterial population in the receiving chamber
    #B_sum_receiver = numpy.sum(numpy.asarray(B_receiver)) # sum of the bacterial population in the receiving chamber
    B_receiver_record.append(B_receiver)
   # B_sum_receiver_record.append(B_sum_receiver)

B_record = numpy.asarray(B_record) # convert B_record to an array
B_receiver_record = numpy.asarray(B_receiver_record) # convert B_receiver to an array
numpy.savetxt("BacterialPopulation.csv", B_record, delimiter=",")
numpy.savetxt("BacterialPopulationReceiver.csv", B_receiver_record, delimiter=",")



In [41]:
# Plot the bacterial population in the heat map
fig6, ax = pyplot.subplots()
pyplot.xlabel('x[m]'); pyplot.ylabel('t[sec]')
heatmap = ax.pcolor(x_grid, t_grid, B_record, vmin=0., vmax=100.)
colorbar = pyplot.colorbar(heatmap)
pyplot.show()
fig6.savefig('Bacterial_population_transient_heatmap_no_convective.png')

In [14]:
# Plot the bacterial population in the last time snap shot
fig7 = pyplot.figure()
pyplot.ylim((0., 100.))
pyplot.xlabel('x[m]'); pyplot.ylabel('Bacterial Population[mM]')
pyplot.plot(x_grid, B)
pyplot.show()
fig7.savefig('Transient_Bacterial_Population.png')


Below I only plot the bacterial population in the receiving chamber!


In [75]:
# Plot the bacterial population (receiving chamber only) in the heat map
fig8, ax = pyplot.subplots()
pyplot.xlabel('x[m]'); pyplot.ylabel('t[sec]')
heatmap = ax.pcolor(x_grid[(second_mem+1):-1], t_grid, B_receiver_record, vmin=0., vmax=100.)
colorbar = pyplot.colorbar(heatmap)
pyplot.show()
fig8.savefig('Receiver_Bacterial_population_transient_heatmap_no_convective.png')

In [42]:
# plot the sum of bacterial population over time 
B_receiver_sum = numpy.sum(B_receiver_record,axis=1) # sum over the rows (spatail grids) in the receiver
B_record_sum = numpy.sum(B_record,axis=1) # sum over the rows (spatial grids) in the whole devices
B_receiver_percentage = B_receiver_sum/B_record_sum*100. 

figsum = pyplot.figure()
pyplot.ylim((0., 100.))
pyplot.xlabel('time[sec]'); pyplot.ylabel('Percentage of the Receiver Bacterial Population[%]')
pyplot.plot(t_grid, B_receiver_percentage)
pyplot.show()
figsum.savefig('Percentage of the Receiver Bacterial Population vs Time')

In [13]:
# Plot the bacterial population (receiving chamber only) in the last time snap shot
fig9 = pyplot.figure()
pyplot.ylim((0., 100.))
pyplot.xlabel('x[m]'); pyplot.ylabel('Receiver Bacterial Population[mM]')
pyplot.plot(x_grid[(second_mem+1):-1], B_receiver)
pyplot.show()
fig9.savefig('Receiver_Transient_Bacterial_Population.png')

toc = time.clock()
elapsed = toc-tic
print elapsed

35.983517
